# Analyse the results

In [ ]:
import os
import numpy as np
from collections import defaultdict

In [ ]:
def list_dir(path):
    return list(filter(lambda f: not f.startswith('.'), os.listdir(path)))

def read_results(path):
    results = []
    with open(path, 'r') as f:
        for l in f.readlines():
            r = float(l)
            results.append(r)
    return results

In [ ]:
return_types = set()
step_sizes_types = set()

results_folder = "results"
seeds = list_dir(results_folder)

results_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

# Structure is: seed/return_function/num_steps/learning_rate.txt
for s in seeds:
    seed_folder = os.path.join(results_folder, str(s))
    returns = list_dir(seed_folder)
    # various return types
    for r in returns:
        return_types.add(r)
        
        returns_folder = os.path.join(seed_folder, str(r))
        step_sizes = list_dir(returns_folder)
        # various step sizes
        for ns in step_sizes:
            step_sizes_types.add(ns)
            
            step_sizes_folder = os.path.join(returns_folder, str(ns))
            files = list_dir(step_sizes_folder)
            # each file has a different learning rate
            for file in files:
                # get the number after 0.
                learning_rate = file.split(".")[1]
                filepath = os.path.join(step_sizes_folder, file)
                results = read_results(filepath)
                
                # we append all the seeds to the same list
                for result in results:
                    results_dict[r][ns][learning_rate].append(result)

In [ ]:
print(results_dict["GAE"])

In [ ]:
print(results_dict["Q"]["1"])

## Find the best learning rate for every step size and return combination

In [ ]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

# print(color.BOLD + 'Hello World !' + color.END)

In [ ]:
for r in return_types:
    best_num_steps = 0
    best_global_mean = 0
    best_global_lr = 0
    
    for ns in step_sizes_types:
        best_lr = 0
        best_mean = 0
        
        for lr in results_dict[r][ns].keys():
            results = np.array(results_dict[r][ns][lr])
            
            mean = np.mean(results)
            median = np.median(results)
            std = np.std(results)
            print(f"Return: {r}\tnum_steps: {ns}\tlr: {lr}\tmean: {mean}\tmedian: {median}\tstd: {std}")
            
            # track which learning rate is best for this num_steps
            if mean > best_mean:
                best_mean = mean
                best_lr = lr

        # track which num_steps is best
        if best_mean > best_global_mean:
            best_num_steps = ns
            best_global_mean = best_mean
            best_global_lr = best_lr
        print(f"Finished for return: {r}, num_steps: {ns}")    
        print(f"Final result for num_steps: {ns}\treturn: {r}\tnum_steps: {ns}\t\tBest lr: {best_lr}\tBest mean: {best_mean}")
    
    print("\n"+"#"*30)    
    print(f"Finished for return: {r}")
    print(color.BOLD+f"Final result for return: {r}\tBest num_steps: {ns}\tBest lr: {best_lr}\tBest mean: {best_mean}"+color.END)
    print("#"*30+"\n")